# Replica del entrenamiento del modelo text2midi

En este notebook se replica todo el proceso de entrenamiento presentado en el paper [Text2midi: Generating Symbolic Music from Captions](http://arxiv.org/abs/2412.16526).

El modelo se puede encontrar perfectamente en hugging face en el [siguiente enlace](https://huggingface.co/amaai-lab/text2midi)

In [1]:
# Global Variables
DEVICE = "cuda" # Choose "cuda" or "cpu" based on your hardware capabilities
DATASET_PATH = "../datasets"

## Preparación del dataset SymphonyNet

Primero se realiza el preprocesamiento del dataset usado para el pre entrenamiento. Se utiliza la librería music 21 para extraer los atributos de tempo, tonalidad, bpm e instrumentos de los archivos midi. Estos atributos sirven como placeholder de 10 frases diferentes, para simplificar usaremos una única plantilla. 

### Prueba de music 21

Primero se prueba que music 21 lea correctamente uno de los archivos midi antes de crear pseudo captions.

In [17]:
# Importar librería music21
from music21 import converter, tempo, key
import os

# Definir la ruta del archivo MIDI
midi_path = "../datasets/SymphonyNet_Dataset/classical/altnikol_befiehl_du_deine_wege_(c)icking-archive.mid"

# Verificar que el archivo existe
if os.path.exists(midi_path):
    print(f"Archivo encontrado: {midi_path}")
else:
    print(f"Archivo NO encontrado: {midi_path}")
    
# Cargar el archivo MIDI
score = converter.parse(midi_path)
print("\nArchivo MIDI cargado correctamente")
print(f"Tipo de objeto: {type(score)}")

Archivo encontrado: ../datasets/SymphonyNet_Dataset/classical/altnikol_befiehl_du_deine_wege_(c)icking-archive.mid

Archivo MIDI cargado correctamente
Tipo de objeto: <class 'music21.stream.base.Score'>

Archivo MIDI cargado correctamente
Tipo de objeto: <class 'music21.stream.base.Score'>


Ahora que sabemos que el archivo se leyó correctamente. Extraemos los atributos requeridos para el pre entrenamiento.

In [18]:
# Mostrar información básica del score
print("=" * 60)
print("INFORMACIÓN BÁSICA DEL ARCHIVO MIDI")
print("=" * 60)
print(f"\nNúmero de partes (tracks): {len(score.parts)}")
print(f"Duración total: {score.quarterLength} quarter notes")
print(f"Duración en segundos: {score.seconds:.2f} segundos")

INFORMACIÓN BÁSICA DEL ARCHIVO MIDI

Número de partes (tracks): 4
Duración total: 1878.0 quarter notes
Duración en segundos: nan segundos


In [23]:
# Crear un resumen estructurado de los atributos extraídos
print("=" * 60)
print("RESUMEN DE ATRIBUTOS EXTRAÍDOS")
print("=" * 60)

# Función auxiliar para extraer atributos de forma robusta
def extract_musical_attributes(score):
    attributes = {
        'tempo_bpm': None,
        'key': None,
        'key_mode': None,
        'time_signature': None,
        'instruments': set(),
        'num_tracks': len(score.parts),
        'duration_seconds': round(score.seconds, 2),
        'num_notes': len(score.flatten().notes)
    }
    
    # Tempo
    tempo_marks = score.flatten().getElementsByClass(tempo.MetronomeMark)
    if tempo_marks:
        attributes['tempo_bpm'] = tempo_marks[0].number
    else:
        try:
            estimated_tempo = score.flatten().metronomeMarkBoundaries()[0][-1]
            attributes['tempo_bpm'] = estimated_tempo.number
        except:
            pass
    
    # Tonalidad
    key_sigs = score.flatten().getElementsByClass(key.Key)
    if key_sigs:
        attributes['key'] = key_sigs[0].tonic.name
        attributes['key_mode'] = key_sigs[0].mode
    else:
        try:
            analyzed_key = score.analyze('key')
            attributes['key'] = analyzed_key.tonic.name
            attributes['key_mode'] = analyzed_key.mode
        except:
            pass
    
    # Compás
    time_sigs = score.flatten().getElementsByClass('TimeSignature')
    if time_sigs:
        attributes['time_signature'] = f"{time_sigs[0].numerator}/{time_sigs[0].denominator}"
    
    # Instrumentos
    for instrument in score.getInstruments() :
        if instrument:
            attributes['instruments'].add(instrument.bestName())

    attributes['instruments'] = ', '.join(attributes['instruments']) if attributes['instruments'] else None
    
    return attributes

# Extraer y mostrar atributos
attributes = extract_musical_attributes(score)

print("\nAtributos extraídos:")
for k, value in attributes.items():
    print(f"  {k}: {value}")

print("\n" + "=" * 60)

RESUMEN DE ATRIBUTOS EXTRAÍDOS

Atributos extraídos:
  tempo_bpm: 120
  key: D
  key_mode: minor
  time_signature: 4/4
  instruments: Pan Flute, Trombone, English Horn, Bass, Alt, Tenor, Violoncello, Sopran
  num_tracks: 4
  duration_seconds: nan
  num_notes: 8139



In [25]:
PSEUDO_TEMPLATE = f"A musical piece in {{key}} {{key_mode}} key with a tempo of {{tempo_bpm}} BPM, time signature of {{time_signature}}, featuring instruments: {{instruments}}."

In [26]:
print(PSEUDO_TEMPLATE.format(**attributes))

A musical piece in D minor key with a tempo of 120 BPM, time signature of 4/4, featuring instruments: Pan Flute, Trombone, English Horn, Bass, Alt, Tenor, Violoncello, Sopran.


Vemos que usando *music21* se obtiene los atributos musicales necesarios para generar las pseudo captions. Esta función se usara más adelante tras iterar por todos los archivos y generar un dataframe con los atributos **location** y **caption** para mantener los nombres del MidiCaps dataset.

### Generar el Dataframe para SymphonyNetDataset

En esta sección se genera el dataframe que sera usado como pre entrenamiento del modelo 

In [29]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

def generate_symphonynet_dataset():
    """
    Recorre todos los archivos MIDI en el dataset SymphonyNet, extrae atributos musicales
    y genera un DataFrame con ubicaciones y captions.
    
    Returns:
        pd.DataFrame: DataFrame con columnas 'location' y 'caption'
    """
    # Inicializar DataFrame
    df = pd.DataFrame(columns=['location', 'caption'])
    
    # Ruta base del dataset
    dataset_path = Path("../datasets/SymphonyNet_Dataset")
    
    # Buscar todos los archivos MIDI
    midi_files = list(dataset_path.rglob("*.mid")) + list(dataset_path.rglob("*.midi"))

    percentage = 0.01
    
    print(f"Se encontraron {len(midi_files)} archivos MIDI")
    print(f"usando {percentage*100}% de los archivos")
    percentage_midi_files = midi_files[:int(len(midi_files)*percentage)]
    print("Procesando archivos...")
    
    # Lista para almacenar los datos temporalmente
    data_rows = []
    errors = []
    
    # Procesar cada archivo MIDI
    for midi_file in tqdm(percentage_midi_files, desc="Procesando archivos MIDI"):
        try:
            # Cargar el archivo MIDI
            score = converter.parse(str(midi_file))
            
            # Extraer atributos musicales
            attributes = extract_musical_attributes(score)
            
            # Generar caption usando la plantilla
            # Manejar valores None en los atributos
            safe_attributes = {
                'key': attributes.get('key') or 'Unknown',
                'key_mode': attributes.get('key_mode') or 'unknown',
                'tempo_bpm': attributes.get('tempo_bpm') or 'Unknown',
                'time_signature': attributes.get('time_signature') or 'Unknown',
                'instruments': attributes.get('instruments') or 'Unknown'
            }
            
            caption = PSEUDO_TEMPLATE.format(**safe_attributes)
            
            # Agregar fila al DataFrame
            data_rows.append({
                'location': str(midi_file.relative_to(dataset_path.parent)),
                'caption': caption
            })
            
        except Exception as e:
            errors.append((str(midi_file), str(e)))
            print(f"\nError procesando {midi_file.name}: {str(e)}")
            continue
    
    # Crear DataFrame a partir de la lista de filas
    df = pd.DataFrame(data_rows)
    
    # Guardar el DataFrame como CSV
    output_path = dataset_path / "symphonynet_captions.csv"
    df.to_csv(output_path, index=False, encoding='utf-8')
    
    print(f"\n{'='*60}")
    print(f"Procesamiento completado!")
    print(f"{'='*60}")
    print(f"Total de archivos procesados exitosamente: {len(df)}")
    print(f"Total de errores: {len(errors)}")
    print(f"Archivo CSV guardado en: {output_path}")
    print(f"{'='*60}")
    
    if errors:
        print(f"\nPrimeros 5 errores:")
        for i, (file, error) in enumerate(errors[:5]):
            print(f"  {i+1}. {Path(file).name}: {error}")
    
    return df

# Ejecutar la función
df_symphonynet = generate_symphonynet_dataset()


Se encontraron 46360 archivos MIDI
usando 1.0% de los archivos
Procesando archivos...


Procesando archivos MIDI:  36%|███▌      | 165/463 [16:33<1:59:00, 23.96s/it]c:\Users\xxraz\OneDrive\Documentos\Desarrollo\maestria\Proyecto\symbolic-music-generation\.venv\lib\site-packages\music21\midi\translate.py:922: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=1, data=b'Fl\xf6ten'>; getting generic Instrument
  warnings.warn(
c:\Users\xxraz\OneDrive\Documentos\Desarrollo\maestria\Proyecto\symbolic-music-generation\.venv\lib\site-packages\music21\midi\translate.py:922: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=1, data=b'Fl\xf6ten'>; getting generic Instrument
  warnings.warn(
c:\Users\xxraz\OneDrive\Documentos\Desarrollo\maestria\Proyecto\symbolic-music-generation\.venv\lib\site-packages\music21\midi\translate.py:922: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, data=b'H\xf6rner'>; getting generic 


Procesamiento completado!
Total de archivos procesados exitosamente: 463
Total de errores: 0
Archivo CSV guardado en: ..\datasets\SymphonyNet_Dataset\symphonynet_captions.csv


In [31]:
# Mostrar las primeras filas del DataFrame generado
print("Primeras 5 filas del dataset:")
display(df_symphonynet.head())

print("\nÚltimas 5 filas del dataset:")
display(df_symphonynet.tail())

print("\nInformación del dataset:")
display(df_symphonynet.info())


Primeras 5 filas del dataset:


,location,caption
0,SymphonyNet_Dataset\classical\altnikol_befiehl...,A musical piece in D minor key with a tempo of...
1,SymphonyNet_Dataset\classical\arriaga_esclavos...,A musical piece in C major key with a tempo of...
2,SymphonyNet_Dataset\classical\arriaga_symphony...,A musical piece in D major key with a tempo of...
3,SymphonyNet_Dataset\classical\arriaga_symphony...,A musical piece in A major key with a tempo of...
4,SymphonyNet_Dataset\classical\arriaga_symphony...,A musical piece in D major key with a tempo of...



Últimas 5 filas del dataset:


,location,caption
458,SymphonyNet_Dataset\classical\mahler_symphony_...,A musical piece in D- major key with a tempo o...
459,SymphonyNet_Dataset\classical\mahler_symphony_...,A musical piece in G major key with a tempo of...
460,SymphonyNet_Dataset\classical\mahler_symphony_...,A musical piece in C major key with a tempo of...
461,SymphonyNet_Dataset\classical\mahler_symphony_...,A musical piece in F major key with a tempo of...
462,SymphonyNet_Dataset\classical\mahler_symphony_...,A musical piece in F major key with a tempo of...



Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   location  463 non-null    object
 1   caption   463 non-null    object
dtypes: object(2)
memory usage: 7.4+ KB


None

En este caso se toma el **1%** del dataset para crear el dataframe. Esto con el objetivo de agilizar el experimento. 

El dataframe se guarda en un archivo csv para poder usar más adelante sin tener que ejecutar nuevamente esta sección.

## Crear el codificador

Ahora se crea el codificador usado en el modelo. Siguiendo la arquitectura presentado en el paper de Text2Midi, se usa [FlanT5](https://huggingface.co/google/flan-t5-base).

In [2]:
from transformers import T5Tokenizer, T5EncoderModel

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

## Load the pre-trained FLAN T5 encoder and freeze its parameters
flan_t5_encoder = T5EncoderModel.from_pretrained("google/flan-t5-small", device_map="auto")
for param in flan_t5_encoder.parameters():
    param.requires_grad = False

input_text = "A musical piece in C major key with a tempo of 120 BPM"
input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(DEVICE)

encoder_outputs = flan_t5_encoder(input_ids)
print(f"Encoder working properly! Output shape: {encoder_outputs.last_hidden_state.shape} (batch_size, seq_len, hidden_dim)")

c:\Users\xxraz\OneDrive\Documentos\Desarrollo\maestria\Proyecto\symbolic-music-generation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes fo

Encoder working properly! Output shape: torch.Size([1, 17, 512]) (batch_size, seq_len, hidden_dim)


## Crear el tokenizador midi

Ahora se crea y valida el tokenizador REMI+. Con esto, el modelo puede procesar correctamente el archivo y las captions.

In [3]:
from miditok import REMI, TokenizerConfig

config = TokenizerConfig(
    use_programs=True,
    use_time_signatures=True,
    one_token_stream_for_programs=True
)

remi_tokenizer = REMI(config)

midi_file_path = "../datasets/SymphonyNet_Dataset/classical/altnikol_befiehl_du_deine_wege_(c)icking-archive.mid"

midi_tokens = remi_tokenizer(midi_file_path)

print(f"Tokenización correcta! Número de tokens: {len(midi_tokens)}")

c:\Users\xxraz\OneDrive\Documentos\Desarrollo\maestria\Proyecto\symbolic-music-generation\.venv\Lib\site-packages\miditok\tokenizations\remi.py:88: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)


Tokenización correcta! Número de tokens: 35160


## Prueba del encoder y los tokenizers

Ahora se prueba que tanto el codificador como el tokenizador se integren adecuadamente con el dataset. De esta forma nos aseguramos que se extraiga correctamente la ubicación del archivo y las captions

In [4]:
import pandas as pd

pretrain_df = pd.read_csv("../datasets/SymphonyNet_Dataset/symphonynet_captions.csv")
pretrain_df.__len__()

463

Ahora que se cargo el dataframe correctamente, se crea una función que retorne la ubicación del archivo midi y su caption respectivo

In [5]:
import os

def get_midi_and_caption(index):
    midi_path = os.path.join(DATASET_PATH, pretrain_df.iloc[index]['location'])
    caption = pretrain_df.iloc[index]['caption']
    return midi_path, caption

midi_path, caption = get_midi_and_caption(0)
print(f"MIDI Path: {midi_path}")
print(f"Caption: {caption}")

MIDI Path: ../datasets\SymphonyNet_Dataset\classical\altnikol_befiehl_du_deine_wege_(c)icking-archive.mid
Caption: A musical piece in D minor key with a tempo of 120 BPM, time signature of 4/4, featuring instruments: Pan Flute, Trombone, English Horn, Bass, Alt, Tenor, Violoncello, Sopran.


Ahora se crea la función que recibe la ubicación del archivo midi y las captions, genera los tokens y los procesa con el encoder

In [6]:
from torch import tensor

def encode_midi_and_caption(midi_path, caption):
    # Tokenizar y codificar la caption
    input_ids = tokenizer(caption, return_tensors="pt", padding=True, truncation=True).input_ids.to(DEVICE)
    encoder_outputs = flan_t5_encoder(input_ids)
    
    # Tokenizar el archivo MIDI
    midi_tokens = remi_tokenizer(midi_path)

    # Convertir los tokens MIDI a tensores
    # ![Importante] asegurarse de que el tensor no sobrepase el limite de tamaño del decoder
    labels = tensor(midi_tokens[:], device=DEVICE)

    return encoder_outputs.last_hidden_state, labels

encoder_outputs, labels = encode_midi_and_caption(midi_path, caption)
print(f"Encoder Output Shape: {encoder_outputs.shape} (batch_size, seq_len, hidden_dim)")
print(f"Labels Shape: {labels.shape} (seq_len,)")

Encoder Output Shape: torch.Size([1, 55, 512]) (batch_size, seq_len, hidden_dim)
Labels Shape: torch.Size([35160]) (seq_len,)


## Crear el decoder

Ahora que verificamos que el dataset se tokeniza y codifica correctamente, se crea el decoder que procesa el resultado del encoder y el midi tokenizado.